In [1]:
import os

## Create directory for each label

In [2]:
COLLECTED_IMG_PATH = r'.\Tensorflow\workspace\images\collectedimages'
IMAGES_PATH = os.path.join('Tensorflow', 'workspace', 'images', 'collectedimages')

labels = os.listdir(COLLECTED_IMG_PATH)

if not os.path.exists(IMAGES_PATH):
    os.mkdir(IMAGES_PATH)
    
for label in labels:
    path = os.path.join(IMAGES_PATH, label)
    if not os.path.exists(path):
        os.mkdir(path)

## Manual Labeling

In [4]:
# !pip install --upgrade pyqt5 lxml

In [5]:
LABELIMG_PATH = os.path.join('Tensorflow', 'labelimg')

if not os.path.exists(LABELIMG_PATH):
    !mkdir {LABELIMG_PATH}
    !git clone https://github.com/tzutalin/labelImg {LABELIMG_PATH}

In [6]:
!cd {LABELIMG_PATH} && pyrcc5 -o libs/resources.py resources.qrc

In [ ]:
# Nanti muncul python gui, pakai python gui itu buat extract fitur
!cd {LABELIMG_PATH} && python labelImg.py

# Splitting Data to Train and Test

In [3]:
TRAIN_PATH = os.path.join('Tensorflow', 'workspace', 'images', 'train')
TEST_PATH = os.path.join('Tensorflow', 'workspace', 'images', 'test')

if not os.path.exists(TRAIN_PATH):
    os.mkdir(TRAIN_PATH)

if not os.path.exists(TEST_PATH):
    os.mkdir(TEST_PATH)

In [8]:
# Shuffle and copy collected image to train and test dir
import random
import shutil
import numpy as np

TRAIN_TO_TOTAL_RATIO = 0.85
RANDOM_SEED = 1

random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

#Creating list for train and test
total = 0
for file_folder_name in labels:
    xml_list = []
    file_for_test = []
    file_dir = os.path.join(IMAGES_PATH, file_folder_name)
    xml_list = np.array([file for file in os.listdir(file_dir) if file.endswith('xml')])
    train_num = int(round(TRAIN_TO_TOTAL_RATIO*len(xml_list)))
    # file_for_training = list(random.sample(xml_list, train_num))
    np.random.shuffle(xml_list)
    file_for_training = xml_list[:train_num]
    file_for_test = [file for file in xml_list if file not in file_for_training]
    
    # Verbose
    name = file_folder_name + '\t' if len(file_folder_name) > 7 else file_folder_name + '\t\t'
    print(f"{name} | Train : {len(file_for_training)}\t| Test : {len(file_for_test)}\t| Total {name} : {len(file_for_training) + len(file_for_test)}")
    total += (len(file_for_training) + len(file_for_test))
    
    #copy file to training
    for file in file_for_training:
        file_xml = os.path.join(file_dir, file)
        file_jpg = os.path.join(file_dir, file[:-3]) + 'jpg'
        jpg_target = os.path.join(TRAIN_PATH, file[:-3]) + 'jpg'
        xml_target = os.path.join(TRAIN_PATH, file)
        
        if not os.path.exists(file_xml):
            print(f'xml ga ada | {file_xml}')
        
        if not os.path.exists(file_jpg): 
            print(f'jpg ga ada | {file_jpg}')
        
        shutil.copy(file_xml, xml_target)
        shutil.copy(file_jpg, jpg_target)
    
    #copy file to test
    for file in file_for_test:
        file_xml = os.path.join(file_dir, file)
        file_jpg = os.path.join(file_dir, file[:-3]) + 'jpg'
        jpg_target = os.path.join(TEST_PATH, file[:-3]) + 'jpg'
        xml_target = os.path.join(TEST_PATH, file)
        
        if not os.path.exists(file_xml):
            print(f'xml ga ada | {file_xml}')
        
        if not os.path.exists(file_jpg): 
            print(f'jpg ga ada | {file_jpg}')
            
        shutil.copy(file_xml, xml_target)
        shutil.copy(file_jpg, jpg_target)

    # break
    # print(f"Train data : {len(os.listdir(TRAIN_PATH)) / 2} data | Test data : {len(os.listdir(TEST_PATH)) / 2} data | Total saat ini : {(len(os.listdir(TRAIN_PATH)) + len(os.listdir(TEST_PATH))) / 2}")
    # print()
    
print(f"Total data : {total}")

botolkaca	 | Train : 765	| Test : 135	| Total botolkaca	 : 900
botolplastik	 | Train : 749	| Test : 132	| Total botolplastik	 : 881
kaleng		 | Train : 915	| Test : 162	| Total kaleng		 : 1077
kardus		 | Train : 586	| Test : 104	| Total kardus		 : 690
karet		 | Train : 499	| Test : 88	| Total karet		 : 587
kertas		 | Train : 620	| Test : 109	| Total kertas		 : 729
plastik		 | Train : 518	| Test : 92	| Total plastik		 : 610
sedotan		 | Train : 969	| Test : 171	| Total sedotan		 : 1140
Total data : 6614


Cek ukuran test dan train

In [5]:
print(f"Train data : {len(os.listdir(TRAIN_PATH)) / 2} data")
print(f"Test data : {len(os.listdir(TEST_PATH)) / 2} data")
print(f"Total data (jumlah split) : {(len(os.listdir(TRAIN_PATH)) + len(os.listdir(TEST_PATH))) / 2}")

Train data : 5620.0 data
Test data : 993.0 data
Total data (jumlah split) : 6613.0


## Cek error kalau ukuran total data != jumlah split

### cek kondisi dataset

In [6]:
total = 0
for file_folder_name in labels:
    file_dir = os.path.join(IMAGES_PATH, file_folder_name)
    data_dir = os.listdir(file_dir)
    jum_jpg, jum_xml = 0, 0
    
    for i in data_dir:
        # print(i[-3:])
        if i[-3:] == 'jpg': 
            jum_jpg += 1
        elif i[-3:] == 'xml':
            jum_xml += 1
        else:
            print('hmmm... sus')

    name = file_folder_name + '\t' if len(file_folder_name) > 7 else file_folder_name + '\t\t'
    print(f"{name} | {jum_jpg * 2} | (jpg, xml, jpg == xml) : {jum_jpg, jum_xml, jum_jpg==jum_xml,} ")
    total += (jum_jpg + jum_xml)
    
print(f'Total : {total / 2}')

botolkaca	 | 1800 | (jpg, xml, jpg == xml) : (900, 900, True) 
botolplastik	 | 1762 | (jpg, xml, jpg == xml) : (881, 881, True) 
kaleng		 | 2154 | (jpg, xml, jpg == xml) : (1077, 1077, True) 
kardus		 | 1380 | (jpg, xml, jpg == xml) : (690, 690, True) 
karet		 | 1174 | (jpg, xml, jpg == xml) : (587, 587, True) 
kertas		 | 1458 | (jpg, xml, jpg == xml) : (729, 729, True) 
plastik		 | 1220 | (jpg, xml, jpg == xml) : (610, 610, True) 
sedotan		 | 2280 | (jpg, xml, jpg == xml) : (1140, 1140, True) 
Total : 6614.0


### cek tiap2 gambar ada xmlnya atau tidak

In [7]:
total = 0
for file_folder_name in labels:
    file_dir = os.path.join(IMAGES_PATH, file_folder_name)
    data_dir = os.listdir(file_dir)
    jum_jpg, jum_xml = 0, 0
    
    isLengkap = True
    for i in range(0, len(data_dir), 2):
        jpg_available = os.path.exists(os.path.join(file_dir, (data_dir[i][:-3] + 'jpg')))
        xml_available = os.path.exists(os.path.join(file_dir, (data_dir[i][:-3] + 'xml')))
        
        if jpg_available != xml_available:
            print(data_dir[i][:-3])
            isLengkap = False 
            
    if isLengkap: 
        print(f"{file_folder_name} lengkap")
    else: 
        print(f"{file_folder_name} tidak lengkap")

botolkaca lengkap
botolplastik lengkap
kaleng lengkap
kardus lengkap
karet lengkap
kertas lengkap
plastik lengkap
sedotan lengkap
